# Mitigating Bias in AI with AIF360
This jupyter notebook is accompanied by a Medium article "bryantruong" wrote, published [here](https://bryantruong3139.medium.com/mitigating-bias-in-ai-with-aif360-b4305d1f88a9): 

### Loading and preparing the dataset

### Data Import
**Importing data from google drive**

In [1]:
# importing from google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [68]:
import pandas as pd
path = "/content/drive/MyDrive/Colab Notebooks/credit_risk.csv"
df = pd.read_csv(path)
# Dataset is now stored in a Pandas Dataframe
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [3]:
!pip install aif360

     |████████████████████████████████| 184kB 23.6MB/s 
     |████████████████████████████████| 358kB 17.4MB/s 
  Created wheel for shap: filename=shap-0.39.0-cp37-cp37m-linux_x86_64.whl size=491627 sha256=7f094c3f3fd7a8bd813c7aa96d5342a0d26136cd89fcfce127826dab698c7e84
  Stored in directory: /root/.cache/pip/wheels/15/27/f5/a8ab9da52fd159aae6477b5ede6eaaec69fd130fa0fa59f283
  Created wheel for memory-profiler: filename=memory_profiler-0.58.0-cp37-none-any.whl size=30188 sha256=f82df6f96014fb17d372c2d22979b8610261beb5447c9ebc85a8425d9b43fb2f
  Stored in directory: /root/.cache/pip/wheels/02/e4/0b/aaab481fc5dd2a4ea59e78bc7231bb6aae7635ca7ee79f8ae5
Successfully built shap memory-profiler


In [4]:
# ModuleNotFoundError: No module named 'fairlearn'; required for aif360
!pip install fairlearn

     |████████████████████████████████| 24.6MB 118kB/s 


In [69]:
# First, read-in the data and check for null values
import numpy as np
import pandas as pd
import aif360
from aif360.algorithms.preprocessing import DisparateImpactRemover
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
pd.options.mode.chained_assignment = None  # default='warn', silencing Setting With Copy warning


In [70]:
# See the different columns and check for null entries
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 981 entries, 0 to 980
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            981 non-null    object 
 1   Gender             957 non-null    object 
 2   Married            978 non-null    object 
 3   Dependents         956 non-null    object 
 4   Education          981 non-null    object 
 5   Self_Employed      926 non-null    object 
 6   ApplicantIncome    981 non-null    int64  
 7   CoapplicantIncome  981 non-null    float64
 8   LoanAmount         954 non-null    float64
 9   Loan_Amount_Term   961 non-null    float64
 10  Credit_History     902 non-null    float64
 11  Property_Area      981 non-null    object 
 12  Loan_Status        981 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 99.8+ KB


In [ ]:
# Let’s take a look at the dataset’s features:
# 1. Loan ID
# 2. Gender (Male or Female)
# 3. Married (Yes or No)
# 4. Dependents (0, 1, 2, or 3+)
# 5. Education (indicating whether or not the primary applicant has graduated from high school)
# 6. Self Employed (Yes or No)
# 7. Applicant Income
# 8. Co-Applicant Income
# 9. Loan Amount
# 10. Loan Applicant Term
# 11. Credit History (0 or 1, with 0 indicating good credit history)
# 12. Property Area (Rural, Semiurban, or Urban)
# 13. Loan Status (Y or N)

In [71]:
# Remove rows with any (even if a single) null values
df = df.dropna(how='any', axis = 0) 
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 769 entries, 1 to 980
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            769 non-null    object 
 1   Gender             769 non-null    object 
 2   Married            769 non-null    object 
 3   Dependents         769 non-null    object 
 4   Education          769 non-null    object 
 5   Self_Employed      769 non-null    object 
 6   ApplicantIncome    769 non-null    int64  
 7   CoapplicantIncome  769 non-null    float64
 8   LoanAmount         769 non-null    float64
 9   Loan_Amount_Term   769 non-null    float64
 10  Credit_History     769 non-null    float64
 11  Property_Area      769 non-null    object 
 12  Loan_Status        769 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 84.1+ KB


The author then wants to check to see the breakdown of values for the outcome variable, `Loan_Status`.

In [72]:
target_counts = df['Loan_Status'].value_counts()
target_counts

#the data seems to be biased as more data points for Y 

Y    561
N    208
Name: Loan_Status, dtype: int64

In [74]:
# Drop unnecessary column
df = df.drop(['Loan_ID'], axis = 1)

KeyError: ignored

### Encode categorical variables: a new way to do it

In [75]:
# Encode Male as 1, Female as 0
df.loc[df.Gender == 'Male', 'Gender'] = 1
df.loc[df.Gender == 'Female', 'Gender'] = 0

# Encode Y Loan_Status as 1, N Loan_Status as 0
df.loc[df.Loan_Status == 'Y', 'Loan_Status'] = 1
df.loc[df.Loan_Status == 'N', 'Loan_Status'] = 0
df

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,1,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,0
2,1,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,1
3,1,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,1
4,1,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,1
5,1,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,1
...,...,...,...,...,...,...,...,...,...,...,...,...
975,1,Yes,1,Graduate,No,2269,2167.0,99.0,360.0,1.0,Semiurban,1
976,1,Yes,3+,Not Graduate,Yes,4009,1777.0,113.0,360.0,1.0,Urban,1
977,1,Yes,0,Graduate,No,4158,709.0,115.0,360.0,1.0,Urban,1
979,1,Yes,0,Graduate,No,5000,2393.0,158.0,360.0,1.0,Rural,0


In [76]:
y = df['Loan_Status']
y

1      0
2      1
3      1
4      1
5      1
      ..
975    1
976    1
977    1
979    0
980    1
Name: Loan_Status, Length: 769, dtype: object

In [77]:
[col for col in df.columns if df[col].dtype in ['object']]

['Gender',
 'Married',
 'Dependents',
 'Education',
 'Self_Employed',
 'Property_Area',
 'Loan_Status']

In [78]:
# Replace the categorical values with the numeric equivalents that we have above
categoricalFeatures = ['Property_Area', 'Married', 'Dependents', 'Education', 'Self_Employed']

# Iterate through the list of categorical features and one hot encode them.
for feature in categoricalFeatures:
    onehot = pd.get_dummies(df[feature], prefix=feature)
    df = df.drop(feature, axis=1)
    df = df.join(onehot)
df

,Gender,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Married_No,Married_Yes,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes
1,1,4583,1508.0,128.0,360.0,1.0,0,1,0,0,0,1,0,1,0,0,1,0,1,0
2,1,3000,0.0,66.0,360.0,1.0,1,0,0,1,0,1,1,0,0,0,1,0,0,1
3,1,2583,2358.0,120.0,360.0,1.0,1,0,0,1,0,1,1,0,0,0,0,1,1,0
4,1,6000,0.0,141.0,360.0,1.0,1,0,0,1,1,0,1,0,0,0,1,0,1,0
5,1,5417,4196.0,267.0,360.0,1.0,1,0,0,1,0,1,0,0,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975,1,2269,2167.0,99.0,360.0,1.0,1,0,1,0,0,1,0,1,0,0,1,0,1,0
976,1,4009,1777.0,113.0,360.0,1.0,1,0,0,1,0,1,0,0,0,1,0,1,0,1
977,1,4158,709.0,115.0,360.0,1.0,1,0,0,1,0,1,1,0,0,0,1,0,1,0
979,1,5000,2393.0,158.0,360.0,1.0,0,1,0,0,0,1,1,0,0,0,1,0,1,0


### Separate dataset by x and y

In [79]:
from sklearn.model_selection import train_test_split
encoded_df = df.copy()
x = df.drop(['Loan_Status'], axis = 1)

### Create Test and Train splits

In [80]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data_std = scaler.fit_transform(x)

# converting to dataframe
data_std = pd.DataFrame(data_std, index=x.index, columns=x.columns)

# We will follow an 80-20 split pattern for our training and test data, respectively
x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.2, random_state = 0)

In [81]:
x.head()

,Gender,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Married_No,Married_Yes,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes
1,1,4583,1508.0,128.0,360.0,1.0,1,0,0,0,1,0,1,0,0,1,0,1,0
2,1,3000,0.0,66.0,360.0,1.0,0,0,1,0,1,1,0,0,0,1,0,0,1
3,1,2583,2358.0,120.0,360.0,1.0,0,0,1,0,1,1,0,0,0,0,1,1,0
4,1,6000,0.0,141.0,360.0,1.0,0,0,1,1,0,1,0,0,0,1,0,1,0
5,1,5417,4196.0,267.0,360.0,1.0,0,0,1,0,1,0,0,1,0,1,0,0,1


In [82]:
data_std.head()

,Gender,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Married_No,Married_Yes,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes
1,0.48205,-0.094784,-0.021068,-0.187346,0.271331,0.421476,1.540392,-0.752457,-0.720946,-0.737683,0.737683,-1.159531,2.302885,-0.461447,-0.318934,0.516610,-0.516610,0.382166,-0.382166
2,0.48205,-0.390107,-0.617811,-1.032088,0.271331,0.421476,-0.649185,-0.752457,1.387067,-0.737683,0.737683,0.862418,-0.434238,-0.461447,-0.318934,0.516610,-0.516610,-2.616666,2.616666
3,0.48205,-0.467902,0.315293,-0.296345,0.271331,0.421476,-0.649185,-0.752457,1.387067,-0.737683,0.737683,0.862418,-0.434238,-0.461447,-0.318934,-1.935695,1.935695,0.382166,-0.382166
4,0.48205,0.169571,-0.617811,-0.010223,0.271331,0.421476,-0.649185,-0.752457,1.387067,1.355595,-1.355595,0.862418,-0.434238,-0.461447,-0.318934,0.516610,-0.516610,0.382166,-0.382166
5,0.48205,0.060807,1.042623,1.706510,0.271331,0.421476,-0.649185,-0.752457,1.387067,-0.737683,0.737683,-1.159531,-0.434238,2.167094,-0.318934,0.516610,-0.516610,-2.616666,2.616666


### Calculating actual disparate impact on testing values from original dataset
Disparate Impact is defined as the ratio of favorable outcomes for the unpriviliged group divided by the ratio of favorable outcomes for the priviliged group.
The acceptable threshold is between .8 and 1.25, with .8 favoring the priviliged group, and 1.25 favoring the unpriviliged group.

In [83]:
y_test.head()

840    1
159    1
148    0
17     0
808    0
Name: Loan_Status, dtype: object

In [85]:
type(y_test)

pandas.core.series.Series

In [99]:
actual_test = x_test.copy()
actual_test['Loan_Status_Actual'] = y_test
print(actual_test.shape)
print(x_test.shape)


(154, 20)
(154, 19)


In [87]:
actual_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 154 entries, 840 to 618
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Gender                   154 non-null    object 
 1   ApplicantIncome          154 non-null    int64  
 2   CoapplicantIncome        154 non-null    float64
 3   LoanAmount               154 non-null    float64
 4   Loan_Amount_Term         154 non-null    float64
 5   Credit_History           154 non-null    float64
 6   Property_Area_Rural      154 non-null    uint8  
 7   Property_Area_Semiurban  154 non-null    uint8  
 8   Property_Area_Urban      154 non-null    uint8  
 9   Married_No               154 non-null    uint8  
 10  Married_Yes              154 non-null    uint8  
 11  Dependents_0             154 non-null    uint8  
 12  Dependents_1             154 non-null    uint8  
 13  Dependents_2             154 non-null    uint8  
 14  Dependents_3+           

In [88]:
actual_test['Gender'].unique()

array([1, 0], dtype=object)

In [92]:
# Priviliged group: Males (1)
# Unpriviliged group: Females (0)
# privilege
male_df = actual_test[actual_test['Gender'].astype('int') == 1]
num_of_priviliged = male_df.shape[0]
print(num_of_priviliged)

# unprivilege
female_df = actual_test[actual_test['Gender'].astype('int') == 0]
num_of_unpriviliged = female_df.shape[0]
print(num_of_unpriviliged)

119
35


In [93]:
unpriviliged_outcomes = female_df[female_df['Loan_Status_Actual'] == 1].shape[0]
unpriviliged_ratio = unpriviliged_outcomes/num_of_unpriviliged
unpriviliged_ratio

0.6

In [39]:
priviliged_outcomes = male_df[male_df['Loan_Status_Actual'] == 1].shape[0]
priviliged_ratio = priviliged_outcomes/num_of_priviliged
priviliged_ratio

0.7226890756302521

In [40]:
# Calculating disparate impact
disparate_impact = unpriviliged_ratio / priviliged_ratio
print("Disparate Impact, Sex vs. Predicted Loan Status: " + str(disparate_impact))

Disparate Impact, Sex vs. Predicted Loan Status: 0.8302325581395349


### Training a model on the original dataset

In [94]:
from sklearn.linear_model import LogisticRegression
# Liblinear is a solver that is very fast for small datasets, like ours
model = LogisticRegression(solver='liblinear', class_weight='balanced')

In [95]:
# y is of type object, so sklearn cannot recognize its type. Add the line y=y.astype('int')
y_train=y_train.astype('int')
model.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

### Evaluating performance

In [96]:
# Let's see how well it predicted with a couple values 
y_pred = pd.Series(model.predict(x_test))
y_test = y_test.reset_index(drop=True)
z = pd.concat([y_test, y_pred], axis=1)
z.columns = ['True', 'Prediction']
z.head()
# Predicts 4/5 correctly in this sample

,True,Prediction
0,1,1
1,1,1
2,0,0
3,0,0
4,0,1


In [97]:
print(type(y_pred))
print(type(y_test))

print(y_pred.dtype)
print(y_test.dtype)

print(y_test.unique())
print(y_pred.unique())
y_pred.head()

print(x_test.shape)

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
int64
object
[1 0]
[1 0]
(154, 19)


In [98]:
import matplotlib.pyplot as plt
from sklearn import metrics
print("Accuracy:", metrics.accuracy_score(y_test.astype('int'), y_pred))
print("Precision:", metrics.precision_score(y_test.astype('int'), y_pred))
print("Recall:", metrics.recall_score(y_test.astype('int'), y_pred))

Accuracy: 0.8116883116883117
Precision: 0.875
Recall: 0.8504672897196262


### Calculating disparate impact on predicted values by model trained on original dataset

In [100]:
# We now need to add this array into x_test as a column for when we calculate the fairness metrics.
y_pred = model.predict(x_test)
x_test['Loan_Status_Predicted'] = y_pred
original_output = x_test
#x_test.drop(['Loan_Status_Predicted'], axis=1, inplace=True)
original_output

,Gender,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Married_No,Married_Yes,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Loan_Status_Predicted
840,1,2553,1768.0,102.0,360.0,1.0,0,0,1,0,1,1,0,0,0,1,0,1,0,1
159,1,4583,5625.0,255.0,360.0,1.0,0,1,0,0,1,1,0,0,0,1,0,1,0,1
148,0,10000,1666.0,225.0,360.0,1.0,1,0,0,1,0,1,0,0,0,1,0,1,0,0
17,0,3510,0.0,76.0,360.0,0.0,0,0,1,1,0,1,0,0,0,1,0,1,0,0
808,1,10000,2690.0,412.0,360.0,1.0,0,1,0,0,1,0,1,0,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611,1,8072,240.0,253.0,360.0,1.0,0,0,1,0,1,0,1,0,0,1,0,1,0,1
471,1,2653,1500.0,113.0,180.0,0.0,1,0,0,0,1,0,1,0,0,0,1,1,0,0
291,1,4400,0.0,127.0,360.0,0.0,0,1,0,0,1,0,0,1,0,1,0,1,0,0
797,0,4000,3917.0,173.0,360.0,1.0,1,0,0,0,1,0,1,0,0,1,0,1,0,0


In [101]:
# Priviliged group: Males (1)
# Unpriviliged group: Females (0)
male_df = original_output[original_output['Gender'] == 1]
num_of_priviliged = male_df.shape[0]
female_df = original_output[original_output['Gender'] == 0]
num_of_unpriviliged = female_df.shape[0]

In [102]:
unpriviliged_outcomes = female_df[female_df['Loan_Status_Predicted'] == 1].shape[0]
unpriviliged_ratio = unpriviliged_outcomes/num_of_unpriviliged
unpriviliged_ratio

0.4857142857142857

In [103]:
priviliged_outcomes = male_df[male_df['Loan_Status_Predicted'] == 1].shape[0]
priviliged_ratio = priviliged_outcomes/num_of_priviliged
priviliged_ratio

0.7310924369747899

In [104]:
# Calculating disparate impact
disparate_impact = unpriviliged_ratio / priviliged_ratio
print("Disparate Impact, Sex vs. Predicted Loan Status: " + str(disparate_impact))

Disparate Impact, Sex vs. Predicted Loan Status: 0.664367816091954


### Applying the Disparate Impact Remover to the dataset

In [105]:
# We are going to be using the dataset with categorical features encoded, encoded_df
encoded_df

,Gender,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Married_No,Married_Yes,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes
1,1,4583,1508.0,128.0,360.0,1.0,0,1,0,0,0,1,0,1,0,0,1,0,1,0
2,1,3000,0.0,66.0,360.0,1.0,1,0,0,1,0,1,1,0,0,0,1,0,0,1
3,1,2583,2358.0,120.0,360.0,1.0,1,0,0,1,0,1,1,0,0,0,0,1,1,0
4,1,6000,0.0,141.0,360.0,1.0,1,0,0,1,1,0,1,0,0,0,1,0,1,0
5,1,5417,4196.0,267.0,360.0,1.0,1,0,0,1,0,1,0,0,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975,1,2269,2167.0,99.0,360.0,1.0,1,0,1,0,0,1,0,1,0,0,1,0,1,0
976,1,4009,1777.0,113.0,360.0,1.0,1,0,0,1,0,1,0,0,0,1,0,1,0,1
977,1,4158,709.0,115.0,360.0,1.0,1,0,0,1,0,1,1,0,0,0,1,0,1,0
979,1,5000,2393.0,158.0,360.0,1.0,0,1,0,0,0,1,1,0,0,0,1,0,1,0


In [109]:
encoded_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 769 entries, 1 to 980
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Gender                   769 non-null    object 
 1   ApplicantIncome          769 non-null    int64  
 2   CoapplicantIncome        769 non-null    float64
 3   LoanAmount               769 non-null    float64
 4   Loan_Amount_Term         769 non-null    float64
 5   Credit_History           769 non-null    float64
 6   Loan_Status              769 non-null    object 
 7   Property_Area_Rural      769 non-null    uint8  
 8   Property_Area_Semiurban  769 non-null    uint8  
 9   Property_Area_Urban      769 non-null    uint8  
 10  Married_No               769 non-null    uint8  
 11  Married_Yes              769 non-null    uint8  
 12  Dependents_0             769 non-null    uint8  
 13  Dependents_1             769 non-null    uint8  
 14  Dependents_2             7

In [110]:
encoded_df.Loan_Status.unique()

array([0, 1], dtype=object)

In [107]:
!pip install BlackBoxAuditing

     |████████████████████████████████| 2.6MB 27.8MB/s 
  Created wheel for BlackBoxAuditing: filename=BlackBoxAuditing-0.1.54-py2.py3-none-any.whl size=1394770 sha256=7fd8ffeb521dd7cc465fa1bd3e3a35c6d6d64c8e969217d819424fb1f727a96d
  Stored in directory: /root/.cache/pip/wheels/66/82/7b/ac2a79b8caf97e15ed415162a7f272cbba1e2e2c851fa76ae3
Successfully built BlackBoxAuditing


In [108]:
import aif360
from aif360.algorithms.preprocessing import DisparateImpactRemover
# binaryLabelDataset = aif360.datasets.BinaryLabelDataset(
#     df=yourDataFrameHere,
#     label_names=['yourOutcomeLabelHere'],
#     protected_attribute_names=['yourProtectedClassHere'])
# Must be a binaryLabelDataset
binaryLabelDataset = aif360.datasets.BinaryLabelDataset(
    favorable_label=1,
    unfavorable_label=0,
    df=encoded_df,
    label_names=['Loan_Status'],
    protected_attribute_names=['Gender'])
di = DisparateImpactRemover(repair_level = 1.0) # require !pip install BlackBoxAuditing
dataset_transf_train = di.fit_transform(binaryLabelDataset)
transformed = dataset_transf_train.convert_to_dataframe()[0] #converted to dataframe
transformed

,Gender,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Married_No,Married_Yes,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Loan_Status
1,1.0,3958.0,1483.0,108.0,360.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,1.0,2600.0,0.0,59.0,360.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
3,1.0,2241.0,2333.0,102.0,360.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
4,1.0,4723.0,0.0,115.0,360.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
5,1.0,4402.0,3683.0,189.0,360.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975,1.0,2101.0,2183.0,79.0,360.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
976,1.0,3719.0,1762.0,94.0,360.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
977,1.0,3762.0,717.0,95.0,360.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
979,1.0,4230.0,2333.0,130.0,360.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


### Train a model using the dataset that underwent the pre-processing

In [111]:
x_trans = transformed.drop(['Loan_Status'], axis = 1)
y = transformed['Loan_Status']

# Liblinear is a solver that is effective for relatively smaller datasets.
model = LogisticRegression(solver='liblinear', class_weight='balanced')
scaler = StandardScaler()
data_std = scaler.fit_transform(x_trans)

# Splitting into test and training
# We will follow an 80-20 split pattern for our training and test data
x_trans_train,x_trans_test,y_trans_train,y_trans_test = train_test_split(x_trans, y, test_size=0.2, random_state = 0)

In [112]:
model.fit(x_trans_train, y_trans_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

### Evaluating performance

In [113]:
# See how well it predicted with a couple values
y_trans_pred = pd.Series(model.predict(x_trans_test))
y_trans_test = y_trans_test.reset_index(drop=True)
z = pd.concat([y_trans_test, y_trans_pred], axis=1)
z.columns = ['True', 'Prediction']
z.head()
# Again, it predicts 4/5 correctly in this sample

,True,Prediction
0,1.0,1.0
1,1.0,1.0
2,0.0,0.0
3,0.0,0.0
4,0.0,1.0


In [116]:
print(y_test.dtype)
print(y_trans_pred.dtype)

object
float64


In [117]:
print("Accuracy:", metrics.accuracy_score(y_test.astype('float'), y_trans_pred))
print("Precision:", metrics.precision_score(y_test.astype('float'), y_trans_pred))
print("Recall:", metrics.recall_score(y_test.astype('float'), y_trans_pred))

Accuracy: 0.8246753246753247
Precision: 0.8846153846153846
Recall: 0.8598130841121495


### Calculating disparate impact on predicted values by model trained on transformed dataset

In [118]:
# We now need to add this array into x_test as a column for when we calculate the fairness metrics.
y_trans_pred = model.predict(x_trans_test)
x_trans_test['Loan_Status_Predicted'] = y_trans_pred
transformed_output = x_trans_test

#x_trans_test.drop(['Loan_Status_Predicted'], axis=1, inplace=True)
transformed_output

,Gender,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Married_No,Married_Yes,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Loan_Status_Predicted
840,1.0,2226.0,1742.0,81.0,360.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
159,1.0,3958.0,5105.0,185.0,360.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
148,0.0,9504.0,1646.0,225.0,300.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
17,0.0,3510.0,0.0,76.0,300.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
808,1.0,10000.0,2541.0,300.0,360.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611,1.0,7600.0,0.0,182.0,360.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
471,1.0,2330.0,1483.0,94.0,180.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
291,1.0,3846.0,0.0,105.0,360.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
797,0.0,4000.0,3917.0,173.0,300.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


Disparate Impact is defined as the ratio of favorable outcomes for the unpriviliged group divided by the ratio of favorable outcomes for the priviliged group. The acceptable threshold is between .8 and 1.25, with .8 favoring the priviliged group, and 1.25 favoring the unpriviliged group.

In [119]:
# Priviliged group: Males (1)
# Unpriviliged group: Females (0)
male_df = transformed_output[transformed_output['Gender'] == 1]
num_of_priviliged = male_df.shape[0]

female_df = transformed_output[transformed_output['Gender'] == 0]
num_of_unpriviliged = female_df.shape[0]

In [120]:
# calculation of unprivileged ratio
unpriviliged_outcomes = female_df[female_df['Loan_Status_Predicted'] == 1].shape[0]
unpriviliged_ratio = unpriviliged_outcomes/num_of_unpriviliged
unpriviliged_ratio

0.5142857142857142

In [121]:
# calculation of privileged ration

priviliged_outcomes = male_df[male_df['Loan_Status_Predicted'] == 1].shape[0]
priviliged_ratio = priviliged_outcomes/num_of_priviliged
priviliged_ratio

0.7226890756302521

In [122]:
# Calculating disparate impact
disparate_impact = unpriviliged_ratio / priviliged_ratio
print("Disparate Impact, Sex vs. Predicted Loan Status: " + str(disparate_impact))

Disparate Impact, Sex vs. Predicted Loan Status: 0.7116279069767442


In [ ]:
# After fixing the bias, we were successfully able to reduce the Disparate Impact, still not in acceptable range